<a href="https://colab.research.google.com/github/rocketsvm/DFDNet/blob/master/DFDNet_DFL_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font size="+4">DFDNet for DeepFaceLab</font></b>

<b><font size="+2">Based on:</font></b>

**GitHub repository**: [DFDNet](https://github.com/csxmli2016/DFDNet)

Article: [Blind Face Restoration via Deep Multi-scale Component Dictionaries](https://arxiv.org/pdf/2008.00418.pdf)

Creator: **[csxmli2016](https://github.com/csxmli2016)**

This Colab notebook based on [notebook by tg-bomze](https://github.com/tg-bomze/DFDNet)

In [1]:
#@title <b><font size="+3">Check GPU</font></b>
!nvidia-smi

Thu Aug 27 10:13:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title <b><font size="+3"> Add shortcut and mount Google Drive</font></b>
#@markdown **Before running this block for the first time for this account you have to make a shortcut to data:**
#@markdown * Follow [this link](https://drive.google.com/drive/folders/1bayYIUMCSGmoFPyd4Uu2Uwn347RW-vl5)
#@markdown * Click on **DFDNet** and select **Add shortcut to Drive**

#@markdown ![](https://raw.githubusercontent.com/tg-bomze/DFDNet/whole/shortcut.JPG)

from google.colab import drive
print("Follow the link below, select the account where you saved the pretrained model,")
print("click the 'Allow' button, copy authorization code in the field below and press Enter.")
drive.mount('/content/drive')

Follow the link below, select the account where you saved the pretrained model,
click the 'Allow' button, copy authorization code in the field below and press Enter.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#@title <b><font size="+3">Prepare Colab machine</font></b>
!pip install dominate

from IPython.display import clear_output
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import os

warnings.filterwarnings("ignore")
%matplotlib inline

!rm -rf /content/sample_data
!git clone --branch master https://github.com/rocketsvm/DFDNet.git

!mkdir /content/DFDNet/checkpoints/
!mkdir /content/DFDNet/checkpoints/facefh_dictionary
!mkdir /content/DFDNet/weights/
!mkdir /content/DFDNet/DictionaryCenter512/

#os.chdir('/content/DFDNet/FaceLandmarkDetection/')
#!python setup.py install
os.chdir('/content/DFDNet/')
!pip install -r requirements.txt
os.chdir('/content/')
!nvidia-smi -L

print("0%/100%   Copying has started and will take about 4 minutes.")
!rsync -ah --progress '/content/drive/My Drive/DFDNet/checkpoints/facefh_dictionary/latest_net_G.pth' '/content/DFDNet/checkpoints/facefh_dictionary'
print("33%/100%  Checkpoints copied")
!rsync -ah --progress  '/content/drive/My Drive/DFDNet/weights/vgg19.pth' '/content/DFDNet/weights/'
print("66%/100%  Weights copied")
!rsync -ah --progress -r '/content/drive/My Drive/DFDNet/DictionaryCenter512/' '/content/DFDNet/DictionaryCenter512/'
print("100%/100% Dictionary copied")

Cloning into 'DFDNet'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 584 (delta 19), reused 28 (delta 6), pack-reused 532
Receiving objects: 100% (584/584), 20.40 MiB | 8.92 MiB/s, done.
Resolving deltas: 100% (200/200), done.
GPU 0: Tesla K80 (UUID: GPU-31b8931e-897b-c5f3-02d5-3a7315a87071)
0%/100%   Copying has started and will take about 4 minutes.
sending incremental file list
latest_net_G.pth
        961.69M 100%   57.77MB/s    0:00:15 (xfr#1, to-chk=0/1)
33%/100%  Checkpoints copied
sending incremental file list
vgg19.pth
        574.67M 100%   17.40MB/s    0:00:31 (xfr#1, to-chk=0/1)
66%/100%  Weights copied
sending incremental file list
./
left_eye_128_center.npy
        209.72M 100%   20.34MB/s    0:00:09 (xfr#1, to-chk=15/17)
left_eye_256_center.npy
        419.43M 100%   17.65MB/s    0:00:22 (xfr#2, to-chk=14/17)
left_eye_32_center.npy
         26.21M 100%  174.61MB/s    0

In [ ]:
#@title <b><font size="+3">Enchance!</font></b>

working_dir = "/content/drive/My Drive/" #@param {type:"string"}

#@markdown Pick a folder from your Google Drive.

#@markdown **working_dir** should contain **aligned** subfolder with JPG images that were extracted by DeepFaceLab *'faceset extract'* scripts.

#@markdown Results will be saved in **aligned_dfdn** subfolder of working_dir.

#@markdown Existing files will be skipped so you can easily continue in case of disconnect.

os.chdir('/content/DFDNet/')
!python -W ignore test_DFL.py --working_dir "$working_dir"

UNetDictFace
model [TestModel] was created
---------- Networks initialized -------------
[Network G] Total number of parameters : 240.117 M
-----------------------------------------------
[1/1242] Restoring 00020.jpg
[2/1242] Restoring 00047.jpg
[3/1242] Restoring 00244.jpg
[4/1242] Restoring 00260.jpg
[5/1242] Restoring 00284.jpg
[6/1242] Restoring 00286.jpg
[7/1242] Restoring 00289.jpg
[8/1242] Restoring 00254.jpg
[9/1242] Restoring 00271.jpg
[10/1242] Restoring 00282.jpg
[11/1242] Restoring 00270.jpg
[12/1242] Restoring 00266.jpg
[13/1242] Restoring 00276.jpg
[14/1242] Restoring 00257.jpg
[15/1242] Restoring 00246.jpg
[16/1242] Restoring 00290.jpg
[17/1242] Restoring 00247.jpg
[18/1242] Restoring 00269.jpg
[19/1242] Restoring 00249.jpg
[20/1242] Restoring 00267.jpg
[21/1242] Restoring 00251.jpg
[22/1242] Restoring 00281.jpg
[23/1242] Restoring 00280.jpg
[24/1242] Restoring 00259.jpg
[25/1242] Restoring 00288.jpg
[26/1242] Restoring 00278.jpg
[27/1242] Restoring 00268.jpg
[28/1242] R